# **Caso de Estudio: Análisis de Ventas de una Cadena de Tiendas**

Contexto:  Supongamos que trabajas como científico de datos para una cadena de tiendas minoristas. Tu tarea es analizar las ventas de productos en diferentes tiendas y generar informes sobre la disponibilidad de productos, así como sobre las tendencias de ventas en distintas regiones.  


Objetivo: Determinar la disponibilidad de productos específicos en diferentes tiendas.
Alinear y analizar las ventas de productos para realizar una comparación entre diferentes regiones y tiendas.  

Datos:  
Productos: Un DataFrame con información sobre productos únicos.  
Ventas: Un DataFrame con registros de ventas diarias.  
Tiendas: Un DataFrame con información sobre las tiendas.  

### Estructura de los Datos

#### Productos

| ProductoID | Nombre      |
|------------|-------------|
| 1          | Laptop      |
| 2          | Smartphone  |
| 3          | Tablet      |
| 4          | Accesorios  |

#### Ventas

| Fecha       | TiendaID | ProductoID | Ventas |
|-------------|----------|------------|--------|
| 2023-01-01  | 101      | 1          | 20     |
| 2023-01-01  | 102      | 2          | 15     |
| 2023-01-01  | 101      | 3          | 10     |
| 2023-01-02  | 101      | 1          | 25     |
| 2023-01-02  | 103      | 2          | 12     |

#### Tiendas

| TiendaID | Región |
|----------|--------|
| 101      | Norte  |
| 102      | Sur    |
| 103      | Este   |
| 104      | Oeste  |

### Uso de `Index.get_indexer`:  
Vamos a utilizar **Index.get_indexer** para alinear los datos de ventas con los datos de productos y tiendas para realizar un análisis detallado.

Paso 1: Crear los dataframes

In [5]:
import pandas as pd
# DataFrame de productos
productos = pd.DataFrame({
    'ProductoID': [1, 2, 3, 4],
    'Nombre': ['Laptop', 'Smartphone', 'Tablet', 'Accesorios']
})

# DataFrame de ventas
ventas = pd.DataFrame({
    'Fecha': ['2023-01-01', '2023-01-01', '2023-01-01', '2023-01-02', '2023-01-02'],
    'TiendaID': [101, 102, 101, 101, 103],
    'ProductoID': [1, 2, 3, 1, 2],
    'Ventas': [20, 15, 10, 25, 12]
})

# DataFrame de tiendas
tiendas = pd.DataFrame({
    'TiendaID': [101, 102, 103, 104],
    'Región': ['Norte', 'Sur', 'Este', 'Oeste']
})

Paso 2. Usar `Index.get_indexer` para Alinear Datos:  
Queremos alinear los datos de ventas con los nombres de productos y las regiones de las tiendas.

In [6]:
producto_index = pd.Index(productos['ProductoID'])
ventas['ProductoIndex'] = producto_index.get_indexer(ventas['ProductoID'])
ventas['NombreProducto'] = productos.loc[ventas['ProductoIndex'], 'Nombre'].values

tienda_index = pd.Index(tiendas['TiendaID'])
ventas['TiendaIndex'] = tienda_index.get_indexer(ventas['TiendaID'])
ventas['Región'] = tiendas.loc[ventas['TiendaIndex'], 'Región'].values

ventas

,Fecha,TiendaID,ProductoID,Ventas,ProductoIndex,NombreProducto,TiendaIndex,Región
0,2023-01-01,101,1,20,0,Laptop,0,Norte
1,2023-01-01,102,2,15,1,Smartphone,1,Sur
2,2023-01-01,101,3,10,2,Tablet,0,Norte
3,2023-01-02,101,1,25,0,Laptop,0,Norte
4,2023-01-02,103,2,12,1,Smartphone,2,Este


### Explicar linea por linea el código utilizado en la celda 5.

### Elabore un breve análisis de los resultados obtenidos. Destacar los principales insights y proponga recomendaciones de mejora para el negocio.

In [12]:
# Contar el número de veces que se vendió cada producto en cada tienda
disponibilidad_productos = ventas.groupby(['NombreProducto', 'TiendaID']).size().unstack(fill_value=0)

# Mostrar la disponibilidad de productos en cada tienda
print("Disponibilidad de Productos en Diferentes Tiendas:")
display(disponibilidad_productos)


Disponibilidad de Productos en Diferentes Tiendas:


TiendaID,101,102,103
NombreProducto,,,
Laptop,2,0,0
Smartphone,0,1,1
Tablet,1,0,0


In [18]:
# Unir ventas y tiendas utilizando left join en 'TiendaID'
ventas_con_region = pd.merge(ventas, tiendas, on='TiendaID', suffixes=('_ventas', '_tiendas'))

# Mostrar las primeras filas del DataFrame unido para verificar
display(ventas_con_region.head())


,Fecha,TiendaID,ProductoID,Ventas,ProductoIndex,NombreProducto,TiendaIndex,Región_x,Nombre_x,Región_y,Nombre_y,Región_ventas,Nombre,Región_tiendas
0,2023-01-01,101,1,20,0,Laptop,0,Norte,Laptop,Norte,Laptop,Norte,Laptop,Norte
1,2023-01-01,102,2,15,1,Smartphone,1,Sur,Smartphone,Sur,Smartphone,Sur,Smartphone,Sur
2,2023-01-01,101,3,10,2,Tablet,0,Norte,Tablet,Norte,Tablet,Norte,Tablet,Norte
3,2023-01-02,101,1,25,0,Laptop,0,Norte,Laptop,Norte,Laptop,Norte,Laptop,Norte
4,2023-01-02,103,2,12,1,Smartphone,2,Este,Smartphone,Este,Smartphone,Este,Smartphone,Este


In [20]:
# Calcular las ventas totales por región y producto
ventas_totales_por_region = ventas_con_region.groupby(['Región_y', 'NombreProducto'])['Ventas'].sum().unstack(fill_value=0)

# Mostrar el análisis de ventas por región
print("\nAnálisis de Ventas por Región:")
display(ventas_totales_por_region)



Análisis de Ventas por Región:


NombreProducto,Laptop,Smartphone,Tablet
Región_y,,,
Este,0,12,0
Norte,45,0,10
Sur,0,15,0


In [21]:
# Unir ventas y tiendas utilizando left join en 'TiendaID'
ventas_con_region = pd.merge(ventas, tiendas, on='TiendaID', how='left', suffixes=('_ventas', '_tiendas'))


In [22]:
# Convertir la columna 'Fecha' a formato datetime
ventas_con_region['Fecha'] = pd.to_datetime(ventas_con_region['Fecha'])


In [24]:
# Calcular las ventas mensuales por región y producto
ventas_con_region['Año-Mes'] = ventas_con_region['Fecha'].dt.to_period('M')
ventas_tendencias_temporales = ventas_con_region.groupby(['Año-Mes', 'Región_y', 'NombreProducto'])['Ventas'].sum().unstack(fill_value=0)

# Mostrar el análisis de tendencias temporales de ventas
print("\nAnálisis de Tendencias Temporales de Ventas por Región y Producto:")
display(ventas_tendencias_temporales)



Análisis de Tendencias Temporales de Ventas por Región y Producto:


NombreProducto    Laptop  Smartphone  Tablet
Año-Mes Región_y                            
2023-01 Este           0          12       0
        Norte         45           0      10
        Sur            0          15       0